In [0]:
%sh
cd /Volumes/workspace/aero2/flights
ls -la
unzip flights.zip
ls -la

In [0]:
from pyspark.sql.functions import col
#user_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
files = ['flights', 'airports', 'airlines']

for f in files:
    file_location = f'/Volumes/workspace/aero2/flights/{f}.csv'
    file_type = "csv"

    # CSV options
    infer_schema = "true"
    first_row_is_header = "true"
    delimiter = ","

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)
    df.write.saveAsTable(f'aero2.{f}')

In [0]:
%sql
show tables from aero2

In [0]:

display(df)

In [0]:
%sql
select * from aero2.flights

In [0]:
%sql
select * from aero2.airlines

In [0]:
%sql
select * from aero2.airports

In [0]:
%sql
select * from aero2.flights order by TaxiIn desc limit 20

In [0]:
%sql
select * from aero2.flights order by TaxiOut desc limit 20

TODO:
1. Find the top 20 shortest flights by Airtime - show where such regular flights are realized 
2. Show these with full name of airports from (Origin) and to (Dest)
3. Add also Latitude and Longitude and visualize these locations of these airports on a map


In [0]:
%sql
CREATE TEMPORARY VIEW shortest_flights as SELECT DISTINCT
    f.Origin,
    f.OriginCityName,
    ao.Latitude AS OriginLat,
    ao.Longitude AS OriginLon,
    f.Dest,
    f.DestCityName,
    ad.Latitude AS DestLat,
    ad.Longitude AS DestLon,
    f.AirTime
FROM aero2.flights f
JOIN aero2.airports ao ON f.Origin = ao.IATA_CODE
JOIN aero2.airports ad ON f.Dest = ad.IATA_CODE
WHERE f.AirTime IS NOT NULL
ORDER BY f.AirTime ASC
LIMIT 20;

In [0]:
%sql
select DISTINCT Origin as Airport, OriginLat as Latitude, OriginLon as Longitude from shortest_flights
UNION 
select distinct Dest as Airport, DestLat as Latitude, DestLon as Longitude from shortest_flights;

Databricks visualization. Run in Databricks to view.

TODO:
1. Count the number of planes
2. Create a list of carriers (full name) with their average arrival delay and number of realized flights. Visualize these delays.
